# Navier Stokes Equatoin

In [1]:
import numpy as np
import time
import scipy.sparse.linalg as sp_la

# --- Core imports ---
from pycutfem.core.mesh import Mesh
from pycutfem.core.dofhandler import DofHandler
from pycutfem.utils.meshgen import structured_quad

# --- UFL-like imports ---
from pycutfem.ufl.functionspace import FunctionSpace
from pycutfem.ufl.expressions import (
    TrialFunction, TestFunction, VectorTrialFunction, VectorTestFunction,
    Function, VectorFunction, Constant, grad, inner, dot, div
)
from pycutfem.ufl.measures import dx
from pycutfem.ufl.forms import BoundaryCondition, assemble_form

# 1. ============================================================================
#    SETUP (Meshes, DofHandler, BCs)
# ===============================================================================
L, H = 1.0, 1.0
NX, NY = 8, 8 # Increased resolution for better visualization
nodes_q2, elems_q2, _, corners_q2 = structured_quad(L, H, nx=NX, ny=NY, poly_order=2)
mesh_q2 = Mesh(nodes=nodes_q2, element_connectivity=elems_q2, elements_corner_nodes=corners_q2, element_type="quad", poly_order=2)

nodes_q1, elems_q1, _, corners_q1 = structured_quad(L, H, nx=NX, ny=NY, poly_order=1)
mesh_q1 = Mesh(nodes=nodes_q1, element_connectivity=elems_q1, elements_corner_nodes=corners_q1, element_type="quad", poly_order=1)

fe_map = {'ux': mesh_q2, 'uy': mesh_q2, 'p': mesh_q1}
dof_handler = DofHandler(fe_map, method='cg')

# Tag boundaries for applying BCs
bc_tags = {
    'bottom_wall': lambda x,y: np.isclose(y,0),
    'left_wall':   lambda x,y: np.isclose(x,0),
    'right_wall':  lambda x,y: np.isclose(x,L),
    'top_lid':     lambda x,y: np.isclose(y,H)
}
mesh_q2.tag_boundary_edges(bc_tags)
mesh_q1.tag_boundary_edges(bc_tags) # Also tag the pressure mesh

# Tag a single node for pressure pinning
mesh_q1.nodes_list[0].tag = 'pressure_pin_point'

class DataBC:
    Um = 1.0 # Lid velocity
    H = H
    
bcs = [
    # No-slip on bottom, left, and right walls
    BoundaryCondition('ux', 'dirichlet', 'bottom_wall', lambda x,y:0.0),
    BoundaryCondition('uy', 'dirichlet', 'bottom_wall', lambda x,y:0.0),
    BoundaryCondition('ux', 'dirichlet', 'left_wall',   lambda x,y:0.0),
    BoundaryCondition('uy', 'dirichlet', 'left_wall',   lambda x,y:0.0),
    BoundaryCondition('ux', 'dirichlet', 'right_wall',  lambda x,y:0.0),
    BoundaryCondition('uy', 'dirichlet', 'right_wall',  lambda x,y:0.0),
    # Moving lid on the top wall
    BoundaryCondition('ux', 'dirichlet', 'top_lid',     lambda x,y:DataBC.Um),
    BoundaryCondition('uy', 'dirichlet', 'top_lid',    lambda x,y:0.0),
    # Pin pressure at one point to ensure a unique solution
    BoundaryCondition('p', 'dirichlet', 'pressure_pin_point',lambda x,y: 0.0)
]

# Create the corresponding homogeneous BCs for the Newton update
bcs_homog = [
    BoundaryCondition(bc.field, bc.method, bc.domain_tag,lambda x,y: 0.0) for bc in bcs
]


In [2]:
# 2. ============================================================================
#    UFL FORMULATION
# ===============================================================================

# --- Define Constants and Function Spaces ---
rho = Constant(1.0)
dt = Constant(0.1)
theta = Constant(0.5) # Crank-Nicolson
mu = Constant(1.0e-2)

velocity_space = FunctionSpace("velocity", ['ux', 'uy'])
pressure_space = FunctionSpace("pressure", ['p'])

# --- Define all required UFL Functions ---
# Trial functions for the Jacobian system (the corrections)
du = VectorTrialFunction(velocity_space)
dp = TrialFunction(pressure_space.field_names[0])

# Test functions
v = VectorTestFunction(velocity_space)
q = TestFunction(pressure_space.field_names[0])

# Functions to hold solutions at different time steps/iterations
# u_k/p_k are the current Newton iteration k at time t_n+1
# u_n/p_n are the converged solution from the previous time step t_n
u_k = VectorFunction(name="u_k", field_names=['ux', 'uy'], dof_handler=dof_handler)
p_k = Function(name="p_k", field_name='p', dof_handler=dof_handler)
u_n = VectorFunction(name="u_n", field_names=['ux', 'uy'], dof_handler=dof_handler)
p_n = Function(name="p_n", field_name='p', dof_handler=dof_handler)

# --- Define Residual R(u_k, p_k) and Jacobian J ---
# The residual is the transient Navier-Stokes equation F(U_k) = 0
# where U_k = (u_k, p_k)

In [3]:


jacobian = (
    # Time derivative term: d/dt(u) -> du/dt
    rho * dot(du, v) / dt  +
    
    # Convection term: theta * [ ((du ⋅ ∇)u_k) ⋅ v + ((u_k ⋅ ∇)du) ⋅ v ]
    theta * rho * dot(du, grad(u_k)) * v +
    theta * rho * dot(u_k, grad(du))* v +

    # Diffusion term: theta * mu * (∇du : ∇v)
    theta * mu * inner(grad(du), grad(v))  -
    
    # Pressure term: -theta * dp * (∇⋅v)
    dp * div(v) +
    
    # Continuity term: q * (∇⋅du)
    q * div(du) 
) * dx()

residual = (
    # Time derivative: -(u_k - u_n)/dt ⋅ v
    -rho * dot(u_k - u_n, v) / dt 

    # Convection terms at theta and (1-theta)
    -(theta * rho * dot(u_k, grad(u_k)) * v ) 
    -((1.0 - theta) * rho * dot(u_n, grad(u_n))* v )
    
    # Diffusion terms at theta and (1-theta)
    -(theta * mu * inner(grad(u_k), grad(v)) ) 
    -((1.0 - theta) * mu * inner(grad(u_n), grad(v)) ) 
    
    # Pressure terms
    -p_k * div(v)  
    
    # Continuity term
    -q * div(u_k) 
    
) *dx()



In [4]:
dof_handler.get_dirichlet_data(bcs_homog[0])

{0: 0.0,
 1: 0.0,
 2: 0.0,
 3: 0.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.0,
 9: 0.0,
 10: 0.0,
 11: 0.0,
 12: 0.0,
 13: 0.0,
 14: 0.0,
 15: 0.0,
 16: 0.0}

In [5]:
# 3. ============================================================================
#    SOLVER LOOP
# ===============================================================================
T_end = 5.0
dt_val = dt.value
num_steps = int(T_end / dt_val)
newton_tol = 1e-6
max_newton_iter = 10

# Apply initial conditions (u=0, p=0)
dof_handler.apply_bcs_to_vector(u_n.nodal_values, bcs)
dof_handler.apply_bcs_to_vector(p_n.nodal_values, bcs)

# --- Main Time-Stepping Loop ---
for n in range(num_steps):
    t = (n + 1) * dt_val
    print(f"\n--- Solving Time Step {n+1}/{num_steps} | t = {t:.2f}s ---")

    # Set initial guess for Newton iteration: u_k = u_n, p_k = p_n
    u_k.nodal_values[:] = u_n.nodal_values[:]
    p_k.nodal_values[:] = p_n.nodal_values[:]
    
    # Apply non-homogeneous BCs to the initial guess for this time step
    dof_handler.apply_bcs_to_vector(u_k.nodal_values, bcs)
    dof_handler.apply_bcs_to_vector(p_k.nodal_values, bcs)

    # --- Inner Newton Iteration Loop ---
    for k in range(max_newton_iter):
        print(f"  Newton iteration {k+1}")

        # Assemble Jacobian matrix A and residual vector R
        # We solve J*dU = -R
        A, R_vec = assemble_form(jacobian == -residual, dof_handler=dof_handler, bcs=bcs_homog)
        
        norm_res = np.linalg.norm(R_vec)
        print(f"    Residual Norm: {norm_res:.3e}")

        if norm_res < newton_tol:
            print(f"    Newton converged in {k+1} iterations.")
            break
        
        # Solve the linear system for the correction dU = (du, dp)
        delta_U = sp_la.spsolve(A, R_vec)
        
        # Update the solution: U_k+1 = U_k + dU
        dof_handler.add_to_functions(delta_U, [u_k, p_k])

    else: # This runs if the for loop completes without breaking
        raise RuntimeError(f"Newton's method did not converge after {max_newton_iter} iterations.")

    # Update solution for the next time step
    u_n.nodal_values[:] = u_k.nodal_values[:]
    p_n.nodal_values[:] = p_k.nodal_values[:]

    # Optional: Save solution to file for visualization
    # u_k.save_to_vtk(f"results/velocity_{n+1:03d}.vtk")
    # p_k.save_to_vtk(f"results/pressure_{n+1:03d}.vtk")

print("\nSimulation finished successfully!")



--- Solving Time Step 1/50 | t = 0.10s ---
  Newton iteration 1
    Residual Norm: 5.594e-02
  Newton iteration 2
    Residual Norm: 9.467e-02
  Newton iteration 3
    Residual Norm: 1.910e-01
  Newton iteration 4
    Residual Norm: 3.948e-01
  Newton iteration 5
    Residual Norm: 1.025e+00
  Newton iteration 6
    Residual Norm: 7.154e+01
  Newton iteration 7
    Residual Norm: 4.647e+05
  Newton iteration 8
    Residual Norm: 5.476e+14
  Newton iteration 9
    Residual Norm: 2.881e+32
  Newton iteration 10
    Residual Norm: 6.900e+76


RuntimeError: Newton's method did not converge after 10 iterations.